#### Check stationary

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series):
    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m") 
        
check_stationarity(df.Close)  
check_stationarity(df.Close.diff(periods=1).dropna()) # diff's p value have to smaller than 10% at least

# creating theis variable
df["close_diff_1"] = df.Close.diff(periods=1)

df.index = df["Date"]
df.drop("Date", axis=1, inplace=True)
# df = date_features(df)

#### Checking Partial Autocorrelation

In [ ]:
plt.rc("figure", figsize=(10,5))
plot_pacf(df['Close'], method='ywm')
plt.show()

# if The data exhibits significant autocorrelation at lag 1. 
# Therefore, we will introduce a feature representing the price lagged by 1.
df["close(-1)"] = df['Close'].shift(1)

from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#### Feature Engineering

In [ ]:
# Lag Features
import pandas as pd
df['lag_1'] = df['close'].shift(1)
df['lag_2'] = df['close'].shift(2)
 #Rolling Statistics
df['rolling_mean'] = df['close'].rolling(window=3).mean()
df['rolling_std'] = df['close'].rolling(window=3).std()

# Seasonal Decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df['close'], model='additive')
decomposition.plot()

import statsmodels.api as sm
# Sample seasonal time series data
seasonal_data = sm.tsa.seasonal_decompose(df['closee'], model='additive', period=4)

# Extract and display the seasonal component
df['seasonal'] = seasonal_data.seasonal

#### Technical indicators

In [ ]:
def SMA(data, window_size):
    return data['Close'].rolling(window=window_size).mean()

def EMA(data, window_size):
    return data['Close'].ewm(span=window_size).mean()

def MACD(data, short_window, long_window):
    short_EMA = EMA(data, short_window)
    long_EMA = EMA(data, long_window)
    return short_EMA - long_EMA

def RSI(data, window_size):
    delta = data['Close'].diff()
    delta = delta[1:] 
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=window_size-1 , min_periods=window_size).mean()
    ema_down = down.ewm(com=window_size-1 , min_periods=window_size).mean()
    return ema_up/ema_down

def Bollinger_Bands(data, window_size):
    middle_band = SMA(data, window_size)
    std_dev = data['Close'].rolling(window=window_size).std()
    upper_band = middle_band + (std_dev*2)
    lower_band = middle_band - (std_dev*2)
    return upper_band, lower_band

df['SMA'] = SMA(df, 13)
df['EMA'] = EMA(df, 9) 
df['MACD'] = MACD(df, 24, 52)
df['RSI'] = RSI(df, 14)
df['Upper_Band'], df['Lower_Band'] = Bollinger_Bands(df, 10)

In [ ]:
# Creating new features
df["H_L_diff"] = df["High"] - df["Low"]
df.drop("Adj Close", axis=1, inplace=True)
df.drop("High", axis=1, inplace=True)
df.drop("Low", axis=1, inplace=True)

df["Bands_diff"] = df["Upper_Band"] - df["Lower_Band"]
df.drop("Upper_Band", axis=1, inplace=True)
df.drop("Lower_Band", axis=1, inplace=True)

df["target"] = df["Close"].shift(-1) # -- regression problem

#### XGboot

In [ ]:
# Feature Engineering
def create_features(df):
    # Calculate daily returns
    df['returns'] = df['Close'].pct_change()
    
    # Lagged returns (past 1, 2, 3 days)
    for lag in [1, 2, 3]:
        df[f'returns_lag_{lag}'] = df['returns'].shift(lag)
    
    # Rolling volatility (standard deviation over 5 days)
    df['volatility_5d'] = df['returns'].rolling(window=5).std()
    
    # Technical indicators: Simple Moving Average (SMA) and Relative Strength Index (RSI)
    df['sma_10'] = df['Close'].rolling(window=10).mean()
    df['sma_20'] = df['Close'].rolling(window=20).mean()
    
    # RSI calculation
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / loss
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    # Time-based features
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    # Drop rows with NaN values
    df = df.dropna()
    return df

#### Train/Test Split

In [ ]:
# Time-Aware Train/Test Split
# Use 80% for training, 20% for testing
train_size = int(0.8 * len(df))
X_train = df[features][:train_size]
y_train = df[target][:train_size]
X_test = df[features][train_size:]
y_test = df[target][train_size:]

#### Model Training

In [ ]:
# Model Training with Walk-Forward Validation
tscv = TimeSeriesSplit(n_splits=5)
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)

#### Forecasting and Evaluation

In [ ]:
rmse_scores, mae_scores, r2_scores, directional_scores = [], [], [], []
for train_idx, val_idx in tscv.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Train model
    model.fit(X_tr, y_tr)
    
    # Predict
    y_pred = model.predict(X_val)
    
    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    # Directional accuracy
    direction_correct = np.mean(np.sign(y_val) == np.sign(y_pred))
    
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)
    directional_scores.append(direction_correct)

#### Residuals Analysis

In [ ]:
residuals = y_test - y_pred_test
plt.figure(figsize=(10, 6))
plt.plot(residuals.index, residuals, label='Residuals')
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals Plot')
plt.legend()
plt.savefig('residuals_plot.png')
plt.close()

# ACF of residuals
acf_values = acf(residuals, nlags=20, fft=True)
plt.figure(figsize=(10, 6))
plt.stem(range(len(acf_values)), acf_values)
plt.title('ACF of Residuals')
plt.savefig('acf_residuals.png')
plt.close()

# Ljung-Box test
ljung_box = sm.stats.acorr_ljungbox(residuals, lags=[10], return_df=True)
print("\nLjung-Box Test for Residuals (lag=10):")
print(ljung_box)

#### Feature Importance

In [ ]:
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance) 

#### Prediction

In [ ]:
# Final Prediction
# Example: Predict next day's return (assuming we have features for the next day)
last_features = df[features].iloc[-1:].copy()
next_return = model.predict(last_features)[0]
print(f"\nPredicted Next Day Return: {next_return:.4f}") 